In [1]:
import numpy as np
# import neuralNetwork 
from os.path  import join
from datasetLoad import MnistDataloader

28


In [2]:
'''
#input layer
class inputLayer:

    def __init__(self, number_of_input): 
        self.number_of_input = number_of_input
    
    #to add the vlaues in the network 
    def use(self, input_matrix):
        if isinstance(input_matrix,np.ndarray):
            self.out_matrix = input_matrix
            return input_matrix
        else:
            self.out_matrix = np.array(input_matrix)
            return self.out_matrix
    
    def fetchInputValues(self):
        return self.out_matrix

#hidden and output layers
class layers:
    
    def __init__(self, number_of_input, number_of_neurons, activation='relu'):
        low, high = -1, 1
        self.weight_matrix = np.random.uniform(low, high, size=(number_of_input, number_of_neurons))
        self.bias = np.random.uniform(low, high, size=(1, number_of_neurons))
        self.activation = activation

    def apply_activation(self, values):
        if self.activation == 'relu':
            return np.maximum(0, values)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-values))
        elif self.activation == 'tanh':
            return np.tanh(values)
        elif self.activation == 'softmax':
            e_x = np.exp(values - np.max(values, axis=-1, keepdims=True))
            return e_x / np.sum(e_x, axis=-1, keepdims=True)
        else:
            return values  # No activation (identity)

    def compute_derivative(self, values):
        if self.activation == 'relu':
            return (values > 0).astype(float)
        elif self.activation == 'sigmoid':
            sigmoid_output = self.apply_activation(values)
            return sigmoid_output * (1 - sigmoid_output)
        elif self.activation == 'tanh':
            tanh_output = self.apply_activation(values)
            return 1 - tanh_output**2
        elif self.activation == 'softmax':
            softmax_output = self.apply_activation(values)
            return softmax_output * (1 - softmax_output)  # Note: This derivative might not be used often for softmax
        else:
            return np.ones_like(values)  # Derivative of the identity function is 1

    def forward(self, input_matrix):
        # print(f'shapeofIN = {np.shape(input_matrix)}, shapeofWI = {np.shape(self.weight_matrix)}') #debugging
        values = np.dot(input_matrix,self.weight_matrix) + self.bias
        self.neuronValues =  self.apply_activation(values)
        # print(self.neuronValues) #debugging
        return self.neuronValues
    
class Model:

    #to intiallize the model
    def __init__(self,):
        inputNumber = int(input('enter the input parameters: ')) # input nodes
        self.numberOfLayers = int(input('enter the number of layers: ')) #number of hidden layer and output layer

        self.listOfHiddenLayers = []
        self.inLayerObj = inputLayer(inputNumber)

        for i in range(self.numberOfLayers):
            number_of_input = int(input('number of input: '))
            numberOfNeurons = int(input('number of neurons: '))
            activation = input('Enter the activation funtion: ')
            hidden_layer_obj = layers(number_of_input,numberOfNeurons,activation)

            self.listOfHiddenLayers.append(hidden_layer_obj)
            
            if i == self.numberOfLayers-1:
                self.numberOfLabels = numberOfNeurons

    def pridict(self,data,ForTraining = False):

        arr = data
        
        for layer in self.listOfHiddenLayers:
            arr = layer.forward(arr)

        if ForTraining:
            return arr

        pridiction_value = -1
        pridiction = None

        for i,obj in enumerate(arr[0]):
            if pridiction_value < obj:
                pridiction = {'label' : i, 'confidence' : obj}
        
        return pridiction
    

    def view_weights(self):
        for layer in self.listOfHiddenLayers:
            print(f'weights \n - {layer.weight_matrix}')
            print(f'bias - {layer.bias}')

    #backpropagation 
    def train(self,data,label,learningRate=0.1):
        
        pridiction = self.pridict(data,ForTraining = True)
        reqArr =np.array([0 for i in range(self.numberOfLabels)])
        reqArr[label] = 1
        costFuntion = sum((pridiction - reqArr)**2)

        pridiction = pridiction - reqArr
        i = self.numberOfLayers - 1

        layers = self.listOfHiddenLayers[i]
        gradient = np.multiply(pridiction,layers.compute_derivative(pridiction))
        prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
        pr = layers.weight_matrix.T - learningRate*(np.dot(gradient.T,prevNuronValues))
        layers.weight_matrix = pr.T
        layers.bias = layers.bias - learningRate*(gradient)
        # print(layers.bias) #debugging
        i -= 1

        while i:
            #to get gradient 
            layers = self.listOfHiddenLayers[i]
            pre_w = self.listOfHiddenLayers[i+1].weight_matrix
            newValues = layers.neuronValues
            # print(f'newValues = {i} - {newValues}') #debugging
            derValues = layers.compute_derivative(newValues)
            cur_gradient = np.multiply(derValues.T,pre_w)
            pr = np.dot(gradient,cur_gradient.T)
            prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
            
            #updating weights
            weights = layers.weight_matrix
            upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
            layers.weight_matrix = upWeights
            gradient = pr
            
            #updating the bias 
            layers.bias = layers.bias - learningRate*(pr)
            # print(layers.bias) #debugging

            i -= 1

        #for input layer - first layer value
        layer = self.listOfHiddenLayers[0]
        pre_w = self.listOfHiddenLayers[1].weight_matrix
        newValues = layers.neuronValue
        derValues = layers.compute_derivative(newValues)
        cur_gradient = np.dot(pre_w,gradient.T)
        pr = np.multiply(derValues,cur_gradient.T)
        prevNuronValues = data
        weights = layer.weight_matrix
        upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
        layers.weight_matrix = upWeights.T
        layers.bias = layers.bias - learningRate*(pr)
    
class ModelTest:

    #to intiallize the model
    def __init__(self,parameters):
        ind = 0
        inputNumber = parameters[ind] # input nodes
        ind += 1
        self.numberOfLayers = parameters[ind] #number of hidden layer and output layer
        ind += 1
        self.listOfHiddenLayers = []
        self.inLayerObj = inputLayer(inputNumber)

        for i in range(self.numberOfLayers):
            number_of_input = parameters[ind]
            ind += 1 
            numberOfNeurons = parameters[ind]
            ind += 1
            activation = parameters[ind]
            ind += 1
            hidden_layer_obj = layers(number_of_input,numberOfNeurons,activation)

            self.listOfHiddenLayers.append(hidden_layer_obj)
            
            if i == self.numberOfLayers-1:
                self.numberOfLabels = numberOfNeurons

    def pridict(self,data,ForTraining = False):

        arr = data
        
        for layer in self.listOfHiddenLayers:
            arr = layer.forward(arr)

        if ForTraining:
            return arr

        pridiction_value = -1
        pridiction = None

        for i,obj in enumerate(arr[0]):
            if pridiction_value < obj:
                pridiction = {'label' : i, 'confidence' : obj}
        
        return pridiction
    

    def view_weights(self):
        for layer in self.listOfHiddenLayers:
            print(f'weights \n - {layer.weight_matrix}')
            print(f'bias - {layer.bias}')

    #backpropagation 
    def train(self,data,label,learningRate=0.1):
        print('here') #debugging
        pridiction = self.pridict(data,ForTraining = True)
        reqArr =np.array([0 for i in range(self.numberOfLabels)])
        reqArr[label] = 1
        costFuntion = sum((pridiction - reqArr)**2)

        pridiction = pridiction - reqArr
        i = self.numberOfLayers - 1
        layers = self.listOfHiddenLayers[i]
        gradient = np.multiply(pridiction,layers.compute_derivative(pridiction))
        # print(f'grad{i} = {gradient.shape}') #debugging
        prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
        # print(f'pre{i} = {prevNuronValues.shape}') #debugging
        pr = layers.weight_matrix.T - learningRate*(np.dot(gradient.T,prevNuronValues))
        print(f'weight{i} = {pr.T.shape}') #debugging
        layers.weight_matrix = pr.T
        layers.bias = layers.bias - learningRate*(gradient)
        # print(layers.bias) #debugging
        i -= 1

        while i:
            #to get gradient 
            layers = self.listOfHiddenLayers[i]
            pre_w = self.listOfHiddenLayers[i+1].weight_matrix
            newValues = layers.neuronValues
            # print(f'newValues = {i} - {newValues}') #debugging
            derValues = layers.compute_derivative(newValues)
            # print(f'der{i} = {derValues.shape}') #debugging
            cur_gradient = np.multiply(derValues.T,pre_w)
            # print(f'grad{i} = {cur_gradient.shape}') #debugging
            pr = np.dot(gradient,cur_gradient.T)
            # print(f'pr{i} = {pr.shape}') #debugging
            prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
            # print(f'prevNu{i} = {prevNuronValues.shape}') #debugging
            
            #updating weights
            weights = layers.weight_matrix
            # print(f'hidden{i} - {np.shape(weights)}') #debugging
            upWeights = weights.T - learningRate*(np.dot(prevNuronValues.T,pr))
            # print(f'wei{i} = {np.shape(upWeights)}') #debugging
            layers.weight_matrix = upWeights
            gradient = pr
            
            #updating the bias 
            layers.bias = layers.bias - learningRate*(pr)
            # print(layers.bias) #debugging
            print(f'weight{i} = {layer.view_weights}') #debugging

            i -= 1

        #for input layer - first layer value
        layer = self.listOfHiddenLayers[0]
        pre_w = self.listOfHiddenLayers[1].weight_matrix
        newValues = self.listOfHiddenLayers[1].neuronValues
        # print(f'new = {np.shape(newValues)}') #debugging
        derValues = layers.compute_derivative(newValues)
        cur_gradient = np.dot(pre_w,gradient.T)
        # print(f'cur_gra = {np.shape(cur_gradient)}') #debugging
        pr = np.multiply(derValues,cur_gradient.T)
        prevNuronValues = data
        weights = layer.weight_matrix
        # print(f'weight = {np.shape(weights)}') #debugging
        upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
        layers.weight_matrix = upWeights.T
        print(f'up{i} = {layer.view_weights}') #debugging
        layers.bias = layers.bias - learningRate*(pr)
'''

"\n#input layer\nclass inputLayer:\n\n    def __init__(self, number_of_input): \n        self.number_of_input = number_of_input\n    \n    #to add the vlaues in the network \n    def use(self, input_matrix):\n        if isinstance(input_matrix,np.ndarray):\n            self.out_matrix = input_matrix\n            return input_matrix\n        else:\n            self.out_matrix = np.array(input_matrix)\n            return self.out_matrix\n    \n    def fetchInputValues(self):\n        return self.out_matrix\n\n#hidden and output layers\nclass layers:\n    \n    def __init__(self, number_of_input, number_of_neurons, activation='relu'):\n        low, high = -1, 1\n        self.weight_matrix = np.random.uniform(low, high, size=(number_of_input, number_of_neurons))\n        self.bias = np.random.uniform(low, high, size=(1, number_of_neurons))\n        self.activation = activation\n\n    def apply_activation(self, values):\n        if self.activation == 'relu':\n            return np.maximum(0

In [3]:


#input layer
class inputLayer:

    def __init__(self, number_of_input): 
        self.number_of_input = number_of_input
    
    #to add the vlaues in the network 
    def use(self, input_matrix):
        if isinstance(input_matrix,np.ndarray):
            self.out_matrix = input_matrix
            return input_matrix
        else:
            self.out_matrix = np.array(input_matrix)
            return self.out_matrix
    
    def fetchInputValues(self):
        return self.out_matrix

#hidden and output layers
class layers:
    
    def __init__(self, number_of_input, number_of_neurons,layerno, activation='relu'):
        low, high = -1, 1
        self.weight_matrix = np.random.uniform(low, high, size=(number_of_input, number_of_neurons))
        self.bias = np.random.uniform(low, high, size=(1, number_of_neurons))
        self.activation = activation
        self.layerNo = layerno

    def apply_activation(self, values):
        if self.activation == 'relu':
            return np.maximum(0, values)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-values))
        elif self.activation == 'tanh':
            return np.tanh(values)
        elif self.activation == 'softmax':
            e_x = np.exp(values - np.max(values, axis=-1, keepdims=True))
            return e_x / np.sum(e_x, axis=-1, keepdims=True)
        else:
            return values  # No activation (identity)

    def compute_derivative(self, values):
        if self.activation == 'relu':
            return (values > 0).astype(float)
        elif self.activation == 'sigmoid':
            sigmoid_output = self.apply_activation(values)
            return sigmoid_output * (1 - sigmoid_output)
        elif self.activation == 'tanh':
            tanh_output = self.apply_activation(values)
            return 1 - tanh_output**2
        elif self.activation == 'softmax':
            softmax_output = self.apply_activation(values)
            return softmax_output * (1 - softmax_output)  # Note: This derivative might not be used often for softmax
        else:
            return np.ones_like(values)  # Derivative of the identity function is 1

    def forward(self, input_matrix):
        # print(f'shapeofIN = {np.shape(input_matrix)}, shapeofWI = {self.weight_matrix.shape} layerNo = {self.layerNo}') #debugging
        values = np.dot(input_matrix,self.weight_matrix) + self.bias
        self.neuronValues =  self.apply_activation(values)
        # print(self.neuronValues) #debugging
        return self.neuronValues
    
class Model:

    #to intiallize the model
    def __init__(self,):
        inputNumber = int(input('enter the input parameters: ')) # input nodes
        self.numberOfLayers = int(input('enter the number of layers: ')) #number of hidden layer and output layer

        self.listOfHiddenLayers = []
        self.inLayerObj = inputLayer(inputNumber)

        for i in range(self.numberOfLayers):
            number_of_input = int(input('number of input: '))
            numberOfNeurons = int(input('number of neurons: '))
            activation = input('Enter the activation funtion: ')
            hidden_layer_obj = layers(number_of_input,numberOfNeurons,activation)

            self.listOfHiddenLayers.append(hidden_layer_obj)
            
            if i == self.numberOfLayers-1:
                self.numberOfLabels = numberOfNeurons
    
    def train(self,data,label,learningRate=0.1):
        pridiction = self.pridict(data,ForTraining = True)
        reqArr =np.array([0 for i in range(self.numberOfLabels)])
        reqArr[label] = 1
        costFuntion = sum((pridiction - reqArr)**2)
        pridiction = pridiction - reqArr
        i = self.numberOfLayers - 1

        layers = self.listOfHiddenLayers[i]
        gradient = np.multiply(pridiction,layers.compute_derivative(pridiction))
        prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
        pr = layers.weight_matrix.T - learningRate*(np.dot(gradient.T,prevNuronValues))
        layers.weight_matrix = pr.T
        layers.bias = layers.bias - learningRate*(gradient)
       
        i -= 1

        while i:
            
            #to get gradient 
            layers = self.listOfHiddenLayers[i]
            pre_w = self.listOfHiddenLayers[i+1].weight_matrix
            newValues = layers.neuronValues
            derValues = layers.compute_derivative(newValues)
            cur_gradient = np.multiply(derValues.T,pre_w)
            pr = np.dot(gradient,cur_gradient.T)
            prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
            
            #updating weights
            weights = layers.weight_matrix
            upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
            layers.weight_matrix = upWeights.T
            gradient = pr
            
            #updating the bias 
            layers.bias = layers.bias - learningRate*(pr)

            i -= 1

        #for input layer - first layer value
        layers = self.listOfHiddenLayers[0]
        pre_w = self.listOfHiddenLayers[1].weight_matrix
        newValues = self.listOfHiddenLayers[1].neuronValues
        derValues = layers.compute_derivative(newValues)
        cur_gradient = np.dot(pre_w,gradient.T)
        pr = np.multiply(derValues,cur_gradient.T)
        prevNuronValues = data
        weights = layers.weight_matrix
        upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
        layers.weight_matrix = upWeights.T
        layers.bias = layers.bias - learningRate*(pr)
        
    
class ModelTest:

    #to intiallize the model
    def __init__(self,parameters):
        ind = 0
        inputNumber = parameters[ind] # input nodes
        ind += 1
        self.numberOfLayers = parameters[ind] #number of hidden layer and output layer
        ind += 1
        self.listOfHiddenLayers = []
        self.inLayerObj = inputLayer(inputNumber)

        for i in range(self.numberOfLayers):
            number_of_input = parameters[ind]
            ind += 1 
            numberOfNeurons = parameters[ind]
            ind += 1
            activation = parameters[ind]
            ind += 1
            hidden_layer_obj = layers(number_of_input,numberOfNeurons,i,activation)
            print(hidden_layer_obj.weight_matrix.shape)
            self.listOfHiddenLayers.append(hidden_layer_obj)
            
            if i == self.numberOfLayers-1:
                self.numberOfLabels = numberOfNeurons

    def pridict(self,data,ForTraining = False):

        arr = data
        
        for layer in self.listOfHiddenLayers:
            arr = layer.forward(arr)
            

        if ForTraining:
            return arr

        pridiction_value = -1
        pridiction = None

        for i,obj in enumerate(arr[0]):
            if pridiction_value < obj:
                pridiction = {'label' : i, 'confidence' : obj}
        
        return pridiction
    

    def view_weights(self):
        for i,layer in enumerate(self.listOfHiddenLayers):
            print(f'layer{i+1}') 
            print(f'weights \n - {layer.weight_matrix}')
            print(f'bias - {layer.bias}')

    def view_neuronValues(self):   
        for i,layer in enumerate(self.listOfHiddenLayers):
            print(f'layer{i+1}') 
            print(f'neuronValues \n - {layer.neuronValues}')

    #backpropagation 
    def train(self,data,label,learningRate=0.1):
        pridiction = self.pridict(data,ForTraining = True)
        reqArr =np.array([0 for i in range(self.numberOfLabels)])
        reqArr[label] = 1
        costFuntion = sum((pridiction - reqArr)**2)

        # print(f'req arr - {reqArr}') #debugging

        print(f'pred = {pridiction}') #debugging
        pridiction = pridiction - reqArr
        i = self.numberOfLayers - 1

        layers = self.listOfHiddenLayers[i]
        gradient = np.multiply(pridiction,layers.compute_derivative(pridiction))
        prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
        pr = layers.weight_matrix.T - learningRate*(np.dot(gradient.T,prevNuronValues))
        layers.weight_matrix = pr.T
        # print(f'we{i}= {np.shape(layers.weight_matrix)}') #debugging
        # print(f'ne{i}= {np.shape(layers.neuronValues)}') #debugging
        layers.bias = layers.bias - learningRate*(gradient)
        # print(f'grad = {gradient}') #debugging
        i -= 1

        while i:
            
            #to get gradient 
            layers = self.listOfHiddenLayers[i]
            pre_w = self.listOfHiddenLayers[i+1].weight_matrix
            newValues = layers.neuronValues
            # print(f'newValues = {i} - {newValues}') #debugging
            derValues = layers.compute_derivative(newValues)
            cur_gradient = np.multiply(derValues.T,pre_w)
            pr = np.dot(gradient,cur_gradient.T)
            # print(f'grad - {pr}') #debugging
            prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
            
            #updating weights
            weights = layers.weight_matrix
            upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
            layers.weight_matrix = upWeights.T
            # print(f'we{i}= {np.shape(layers.weight_matrix)}') #debugging
            # print(f'ne{i}= {np.shape(layers.neuronValues)}') #debugging
            gradient = pr
            
            #updating the bias 
            layers.bias = layers.bias - learningRate*(pr)
            # print(layers.bias) #debugging

            i -= 1

        #for input layer - first layer value
        layers = self.listOfHiddenLayers[0]
        pre_w = self.listOfHiddenLayers[1].weight_matrix
        newValues = self.listOfHiddenLayers[1].neuronValues
        derValues = layers.compute_derivative(newValues)
        cur_gradient = np.dot(pre_w,gradient.T)
        pr = np.multiply(derValues,cur_gradient.T)
        # print(f'grad - {pr}') #debugging
        prevNuronValues = data
        weights = layers.weight_matrix
        upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
        layers.weight_matrix = upWeights.T
        # print(f'we{i}= {np.shape(layers.weight_matrix)}') #debugging
        # print(f'ne{i}= {np.shape(layers.neuronValues)}') #debugging
        layers.bias = layers.bias - learningRate*(pr)


In [4]:
input_path = r'C:\programs\projects\NN\input'
training_images_filepath = join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')


mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

x_train = np.array(x_train)
x_test = np.array(x_test)

# print(x_test[1],y_test[1])

In [5]:
# Flatten the training and test images
x_train_flat = x_train.reshape(x_train.shape[0], -1)/255  
x_test_flat = x_test.reshape(x_test.shape[0], -1)     

testInputs = [784 ,3,784,16,'relu',16,16,'relu',16,10,'sigmoid']
model = ModelTest(testInputs)

'''
input parameters = 784
number of layers = 3

layer 1:
number of input: 784
number of neurons: 16
Enter the activation funtion: relu

layer 2:
number of input: 16
number of neurons: 16
Enter the activation funtion: relu

layer 3:
number of input: 16
number of neurons: 10
Enter the activation funtion: sigmoid
'''

(784, 16)
(16, 16)
(16, 10)


'\ninput parameters = 784\nnumber of layers = 3\n\nlayer 1:\nnumber of input: 784\nnumber of neurons: 16\nEnter the activation funtion: relu\n\nlayer 2:\nnumber of input: 16\nnumber of neurons: 16\nEnter the activation funtion: relu\n\nlayer 3:\nnumber of input: 16\nnumber of neurons: 10\nEnter the activation funtion: sigmoid\n'

In [6]:
for x, y in zip(x_train_flat, y_train[:3]):
    x = np.array([x])
    # model.view_weights()
    model.train(x, y)
    model.view_neuronValues()


pred = [[1.31682017e-18 9.99999997e-01 3.89699941e-13 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  9.99961735e-01 1.00000000e+00]]
layer1
neuronValues 
 - [[ 0.          0.          3.12997168  5.28690096  9.03985486  3.23132037
  19.64828554  4.24926767  0.          0.          3.77605531  4.09499052
   0.          0.          0.          0.        ]]
layer2
neuronValues 
 - [[22.83976792 19.85484559  6.31842221  0.          0.9645683   0.48994338
   0.          0.          0.         19.82246016  8.81888101 24.46529089
   0.         14.21822934 26.50244999  7.91290158]]
layer3
neuronValues 
 - [[1.31682017e-18 9.99999997e-01 3.89699941e-13 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  9.99961735e-01 1.00000000e+00]]
pred = [[1.73115296e-24 5.79436067e-13 1.70122150e-07 3.57133486e-20
  3.51226513e-14 1.00000000e+00 1.74867845e-04 9.85990459e-23
  1.49041591e-31 3.91890541e-14]]
layer1
neuronValues 
 - [[ 0.          0

In [7]:
a = np.array([x_train_flat[0]])
# print(model.pridict(a,True))

print(a)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.   